# Import necessary lib. 


In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow.contrib.layers import l1_l2_regularizer

# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__)

import os
import h5py
import time
# from batchnorm import batch_norm,fc_batch_norm

from sklearn.metrics import accuracy_score,classification_report
import Model_1229 as my_model

E:\Liulicong\Software\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.12.0


# Configuration
Set the class names and num (17).
Set the data path (relative path). 

In [2]:
# configuration
# data attribution
CLASS_NUM = 17
CLASS_URBAN = ['Urban_Compact high-rise', 'Urban_Compact mid-rise', 'Urban_Compact low-rise',
               'Urban_Open high-rise', 'Urban_Open mid-rise', 'Urban_Open low-rise',
               'Urban_Lightweight low-rise', 'Large low-rise', 'Sparesly built', 'Heavy industry']
CLASS_RURAL = ['Dense trees', 'Scattered trees', 'Bush and scrub', 'Low plants', 'Bare rock or paved', 'Bare soil or sand', 'Water']
CLASS_NAME = CLASS_URBAN + CLASS_RURAL

# data path
base_dir = r'../data/'
path_training = os.path.join(base_dir, 'training.h5')
path_validation = os.path.join(base_dir, 'validation.h5')
path_test = os.path.join(base_dir, 'round1_test_a_20181109.h5')
# random
np.random.seed(0)

# Read Data
Check the data foramt.

In [3]:
h5py_training = h5py.File(path_training,'r')
h5py_validation = h5py.File(path_validation,'r')
h5py_test = h5py.File(path_test, 'r')

### get s1 image channel data
### it is not really loaded into memory. only the indexes have been loaded.
print("-" * 60)
print("training part")
s1_training = h5py_training['sen1']
print('s1_training: ', s1_training.shape)
s2_training = h5py_training['sen2']
print('s2_training: ', s2_training.shape)
label_training = h5py_training['label']
print('label_training: ', label_training.shape)

print("-" * 60)
print("validation part")
s1_validation = h5py_validation['sen1']
print('s1_validation: ', s1_validation.shape)
s2_validation = h5py_validation['sen2']
print('s2_validation: ', s2_validation.shape)
label_validation = h5py_validation['label']
print('label_validation: ', label_validation.shape)

print("-" * 60)
print("Test part")
s1_test = h5py_test['sen1']
print('s1_test: ', s1_test.shape)
s2_test = h5py_test['sen2']
print('s2_test: ', s2_test.shape)


train_num, pixel_x, pixel_y, sar_bands = s1_training.shape
vali_num, pixel_x, pixel_y, optical_bands = s2_validation.shape
test_num = s1_test.shape[0]

------------------------------------------------------------
training part
s1_training:  (352366, 32, 32, 8)
s2_training:  (352366, 32, 32, 10)
label_training:  (352366, 17)
------------------------------------------------------------
validation part
s1_validation:  (24119, 32, 32, 8)
s2_validation:  (24119, 32, 32, 10)
label_validation:  (24119, 17)
------------------------------------------------------------
Test part
s1_test:  (4838, 32, 32, 8)
s2_test:  (4838, 32, 32, 10)


# Pre Training

In [6]:
PRE_MODEL_prefix = "1229_PRETRAINING"
PRE_MODEL_name = "{}".format(PRE_MODEL_prefix)
my_model.build_model(PRE_MODEL_name)
my_model.training((s2_training, label_training), (s2_validation, label_validation), 
                  model_name=PRE_MODEL_name, epochs=10)

Model Build Start!
Model Build Success!
epochs : 0 Valid With AC: 0.6162136256202193 With Loss: 1.3054363997729195
epochs : 1 Valid With AC: 0.638200571714354 With Loss: 1.3769340254170241
epochs : 2 Valid With AC: 0.6339145066324344 With Loss: 1.4240321337254942
epochs : 3 Valid With AC: 0.5971886973735715 With Loss: 1.603286709001178
epochs : 4 Valid With AC: 0.6034828818534032 With Loss: 1.6579199956668305
epochs : 5 Valid With AC: 0.6118730449479474 With Loss: 1.6321172713837995
epochs : 6 Valid With AC: 0.6087905395129496 With Loss: 1.7279509776830426
epochs : 7 Valid With AC: 0.5876467131878719 With Loss: 1.7783923259624237
epochs : 8 Valid With AC: 0.6098350596821998 With Loss: 1.9469587784006435
epochs : 9 Valid With AC: 0.59836065030295 With Loss: 1.8746792566984356
Training time using: 807.9498949050903


# One Model Training

In [49]:
ONE_MODEL_prefix = "1229_ONEMODEL"
ONE_MODEL_name = "{}".format(PRE_MODEL_prefix)
my_model.build_model(PRE_MODEL_name)
my_model.training((s2_validation[5000:, :, :, :], label_validation[5000:,:]), 
                  (s2_validation[:5000, :, :, :], label_validation[:5000,:]), 
                  model_name=ONE_MODEL_name, epochs=50, early_stop=5, is_shuffle=True)

Model Build Start!
Model Build Success!
epochs : 0 Valid With AC: 0.11339999973773957 With Loss: 5.050211242675781
epochs : 1 Valid With AC: 0.11339999973773957 With Loss: 5.938496801757813
epochs : 2 Valid With AC: 0.12200000077486038 With Loss: 5.096493896484375
epochs : 3 Valid With AC: 0.18259999960660933 With Loss: 4.234229638671875
epochs : 4 Valid With AC: 0.49139999866485595 With Loss: 1.911457586669922
epochs : 5 Valid With AC: 0.7699999976158142 With Loss: 0.7632871765136718
epochs : 6 Valid With AC: 0.8138000035285949 With Loss: 0.6028267807006836
epochs : 7 Valid With AC: 0.8446000003814698 With Loss: 0.47860005493164065
epochs : 8 Valid With AC: 0.8738000011444091 With Loss: 0.38220713348388674
epochs : 9 Valid With AC: 0.8416000056266785 With Loss: 0.46222345123291014
epochs : 10 Valid With AC: 0.8867999958992004 With Loss: 0.35815686492919924
epochs : 11 Valid With AC: 0.884399995803833 With Loss: 0.35007618560791015
epochs : 12 Valid With AC: 0.8778000020980835 With Los

# CV Training

In [5]:
from imp import reload
reload(my_model)

<module 'Model_1229' from 'E:\\Liulicong\\Kaggle\\sentinel\\src\\Model_1229.py'>

In [51]:
MODEL_Prefix = "SHUFFLE_FALSE_New_Model"
N = 5
kflod_turns = 1

In [52]:
from sklearn.model_selection import StratifiedKFold
for kflod_turn in range(kflod_turns):
    sfolder = StratifiedKFold(n_splits=N, random_state=kflod_turn, shuffle=False)
    for i, (train_loc, vali_loc) in enumerate(sfolder.split(np.zeros((vali_num, 1)), np.zeros(vali_num))):
        _id = i + N * kflod_turn
        print(_id, '; Train: %s | test: %s' % (train_loc, vali_loc))
        
        X_optical_train = s2_validation[train_loc, :, :, :]
        y_train = label_validation[train_loc, :]

        X_optical_vali = s2_validation[vali_loc, :, :, :]
        y_vali = label_validation[vali_loc, :]
        
        my_model.build_model("{}{}".format(MODEL_Prefix, _id))
        my_model.training((X_optical_train, y_train), (X_optical_vali, y_vali), 
                          model_name="{}{}".format(MODEL_Prefix, _id), epochs=50, 
                          early_stop=5, is_shuffle=True)

0 ; Train: [ 4824  4825  4826 ... 24116 24117 24118] | test: [   0    1    2 ... 4821 4822 4823]
Model Build Start!
Model Build Success!
epochs : 0 Valid With AC: 0.11359999984502793 With Loss: 3.8754233618083087
epochs : 1 Valid With AC: 0.11359999984502793 With Loss: 4.2893056853691345
epochs : 2 Valid With AC: 0.12639999985694886 With Loss: 4.733606597875085
epochs : 3 Valid With AC: 0.20546666622161866 With Loss: 3.4360282765099064
epochs : 4 Valid With AC: 0.5666666686534881 With Loss: 1.5315565801971587
epochs : 5 Valid With AC: 0.7220000004768372 With Loss: 0.8963925047695736
epochs : 6 Valid With AC: 0.8451333332061768 With Loss: 0.48595711583917217
epochs : 7 Valid With AC: 0.8481333374977111 With Loss: 0.4634843501285534
epochs : 8 Valid With AC: 0.7772000026702881 With Loss: 0.7907963534492758
epochs : 9 Valid With AC: 0.8447333288192749 With Loss: 0.4425640551011954
epochs : 10 Valid With AC: 0.8795333337783814 With Loss: 0.3459531209164393
epochs : 11 Valid With AC: 0.8755

epochs : 10 Valid With AC: 0.8915217399597168 With Loss: 0.3344069136244465
epochs : 11 Valid With AC: 0.884660861492157 With Loss: 0.34283662934830245
epochs : 12 Valid With AC: 0.8890608716011047 With Loss: 0.3649977060266978
epochs : 13 Valid With AC: 0.8978608679771424 With Loss: 0.30171993968646454
epochs : 14 Valid With AC: 0.9030608654022216 With Loss: 0.2876801576798273
epochs : 15 Valid With AC: 0.9109999942779541 With Loss: 0.28067103995343545
epochs : 16 Valid With AC: 0.9043826103210449 With Loss: 0.2978753098407376
epochs : 17 Valid With AC: 0.9035826015472412 With Loss: 0.2820307363644196
epochs : 18 Valid With AC: 0.9007999992370606 With Loss: 0.32467139573011217
epochs : 19 Valid With AC: 0.9218608736991882 With Loss: 0.23901243815895631
epochs : 20 Valid With AC: 0.9032608699798584 With Loss: 0.3098802238288767
epochs : 21 Valid With AC: 0.9178608703613281 With Loss: 0.24032863338730678
epochs : 22 Valid With AC: 0.9126608633995056 With Loss: 0.2889931313160567
epochs 

# CV Test 

In [57]:
MODEL_Prefix = "SHUFFLE_FALSE_New_Model"

In [58]:
CV_proba = np.zeros((test_num, CLASS_NUM))
for i in range(N * kflod_turns):
    my_model.build_model("{}{}".format(MODEL_Prefix, i))
    CV_proba += my_model.batch_predict(s2_test, "{}{}".format(MODEL_Prefix, i))

Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ../ckpt/SHUFFLE_FALSE_New_Model0.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ../ckpt/SHUFFLE_FALSE_New_Model1.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ../ckpt/SHUFFLE_FALSE_New_Model2.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ../ckpt/SHUFFLE_FALSE_New_Model3.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ../ckpt/SHUFFLE_FALSE_New_Model4.ckpt


In [59]:
CV_proba_max_loc = np.argmax(CV_proba, 1)
one_hot=tf.one_hot(CV_proba_max_loc,CLASS_NUM)
with tf.Session() as sess:
    _one_hot = sess.run(one_hot)
_one_hot = _one_hot.astype(np.int16)
pd.DataFrame(_one_hot).to_csv('../result/1229_deep_result_new{}.csv'.format(N), index=False, header=None)